In [2]:
#!pip install rpy2
####
# Non-Exported Utility Functions
####

#' Calculate FREX (FRequency and EXclusivity) Words
#' 
#' A primarily internal function for calculating FREX words.
#' We expect most users will use \code{\link{labelTopics}} instead.
#' 
#' FREX attempts to find words which are both frequent in and exclusive to a topic of interest.
#' Balancing these two traits is important as frequent words are often by themselves simply functional
#' words necessary to discuss any topic.  While completely exclusive words can be so rare as to not
#' be informative. This accords with a long-running trend in natural language processing which is best exemplified
#' by the Term frequency-Inverse document frequency metric.  
#' 
#' Our notion of FREX comes from a paper by Bischof and Airoldi (2012) which proposed a Hierarchical
#' Poisson Deconvolution model.  It relies on a known hierarchical structure in the documents and requires
#' a rather complicated estimation scheme.  We wanted a metric that would capture their core insight but
#' still be fast to compute.
#' 
#' Bischof and Airoldi consider as asummary for a word's contribution to a topic the harmonic mean of the
#' word's rank in terms of exclusivity and frequency.  The harmonic mean is attractive here because it 
#' does not allow a high rank along one of the dimensions to compensate for the lower rank in another. Thus
#' words with a high score must be high along both dimensions.
#' 
#' The formula is ' 
#'\deqn{FREX = \left(\frac{w}{F} + \frac{1-w}{E}\right)^{-1}}{FREX = ((w/F) + ((1-w)/E))^-1} 
#' where F is the frequency score given by the emperical CDF of the word in it's topic distribution.  Exclusivity
#' is calculated by column-normalizing the beta matrix (thus representing the conditional probability of seeing
#' the topic given the word).  Then the empirical CDF of the word is computed within the topic.  Thus words with
#' high values are those where most of the mass for that word is assigned to the given topic.
#' 
#' For rare words exclusivity will always be very high because there simply aren't many instances of the word.
#' If \code{wordcounts} are passed, the function will calculate a regularized form of this distribution using a
#' James-Stein type estimator described in \code{\link{js.estimate}}.
#' 
#' @param logbeta a K by V matrix containing the log probabilities of seeing word v conditional on topic k
#' @param w a value between 0 and 1 indicating the proportion of the weight assigned to frequency 
#' @param wordcounts a vector of word counts.  If provided, a James-Stein type shrinkage estimator is 
#' applied to stabilize the exclusivity probabilities. This helps with the concern that the rarest words
#' will always be completely exclusive.
#' @references 
#' Bischof and Airoldi (2012) "Summarizing topical content with word frequency and exclusivity"
#' In Proceedings of the International Conference on Machine Learning.
#' @seealso \code{\link{labelTopics}} \code{\link{js.estimate}}
#' @export
#' @keywords internal
!pip3 install scipy

  Using cached https://files.pythonhosted.org/packages/7f/5f/c48860704092933bf1c4c1574a8de1ffd16bf4fde8bab190d747598844b2/scipy-1.2.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f5/bf/4981bcbee43934f0adb8f764a1e70ab0ee5a448f6505bd04a87a2fda2a8b/numpy-1.16.1-cp36-cp36m-manylinux1_x86_64.whl


In [3]:
!pip3 install https://s3.amazonaws.com/h2o-release/datatable/stable/datatable-0.8.0/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl

  Using cached https://s3.amazonaws.com/h2o-release/datatable/stable/datatable-0.8.0/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3f/96/1915827a8e411613d364dd3a56ef1fbfab84ee878070a69c21b10b5ad1bb/blessed-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/37/3757249f05aac8a08d9742f9a35c17ab6895eb916b83bbf3a23eae6842b2/typesentry-0.2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/9f/526a6947247599b084ee5232e4f9190a38f398d7300d866af3ab571a5bfe/wcwidth-0.1.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4f/a6/728666f39bfff1719fc94c481890b2106837da9318031f71a8424b662e12/colorama-0.4.1-py2.py3-none-any.whl


In [4]:
import numpy as np
import pandas as pd
from scipy import special as sp
from scipy import stats as ss
import datatable as dt

In [5]:

def get_col(arr, col):
    return map(lambda x : x[col], arr)


In [6]:
#' Accurately computes the logarithm of the sum of exponentials, that is,

#' \eqn{log(sum(exp(lx)))}.

#It is a smooth approximation of max[\vec{x}_j]

def col_lse(mat): 
    return sp.logsumexp(mat, axis=0)
    

In [7]:

#Testing col_lse (matches with R output)
print(col_lse(np.identity(3)))
print(col_lse(np.array([[2,3,4],[4,5,7],[10,11,111]]).T))
print(col_lse(np.array([[.7, .5, -70,0,1],[0,5,80,1,1],[2.7,3000,-7,0,-5000],[0,0,0,1,0],[-1,-2,-3,-4,-58]]).T))

[1.55144471 1.55144471 1.55144471]
[  4.40760596   7.16984602 111.        ]
[ 1.99887605e+00  8.00000000e+01  3.00000000e+03  1.90483244e+00
 -5.59810301e-01]


In [8]:
'''js.estimate <- function(prob, ct) {
  if(ct<=1) {
    #basically if we only observe a count of 1
    #the variance goes to infinity and we get the uniform distribution.
    return(rep(1/length(prob), length(prob)))
  }
  # MLE of prob estimate
  mlvar <- prob*(1-prob)/(ct-1)
  unif <- rep(1/length(prob), length(prob)) 
  
  # Deviation from uniform
  deviation <- sum((prob-unif)^2)
  
  #take care of special case,if no difference it doesn't matter
  if(deviation==0) return(prob)
  
  lambda <- sum(mlvar)/deviation
  #if despite  our best efforts we ended up with an NaN number-just return the uniform distribution.
  if(is.nan(lambda)) return(unif)
  
  #truncate
  if(lambda>1) lambda <- 1
  if(lambda<0) lambda <- 0
  
  #Construct shrinkage estimator as convex combination of the two
  lambda*unif + (1 - lambda)*prob
}
'''
#JSestimate: np.array representation of probability distribution -> np.array representation of estimated prob


def JSestimate(prob, ct):
    #basically if we only observe a count of 1
    #the variance goes to infinity and we get the uniform distribution.
    if (ct <= 1):
         
        return (1.0/np.shape(prob))*np.ones(np.shape(prob))
    #Find the MLE for prob estimate
    mlvar = prob*(1-prob)/(ct-1)
    unif = (1.0/np.shape(prob))*np.ones(np.shape(prob))
    
    #We measure the deviation from uniform
    dev = sum((prob-unif)**2)
    if (dev == 0):
        return prob
    xi = sum(mlvar)/dev
      #if despite  our best efforts we ended up with an NaN number-just return the uniform distribution.
    if (np.isnan(xi)): 
        return(unif)
    #Truncate
    if (xi>1):
        xi = 1
    if (xi < 0):
        xi = 0
    #Construct shrinkage estimator as convex combination of the two
    return (xi*unif) + (1-xi)*prob     

In [9]:
(np.array([1,2,3])-np.ones((3,3)))

array([[0., 1., 2.],
       [0., 1., 2.],
       [0., 1., 2.]])

In [10]:
#coding safelog()
#R code-

'''

safelog <- function(x, min=-1000) {
 out <- log(x)
 out[which(out<min)] <- min
 out

}
'''

def safelog(x, min =-1000):
    output = np.log(x)
    output[np.where(output<min)] = min
    return output


In [11]:
#
'''
calcfrex <- function(logbeta, w=.5, wordcounts=NULL) {
  excl <- t(t(logbeta) - col.lse(logbeta))
  if(!is.null(wordcounts)) {
    #if word counts provided calculate the shrinkage estimator
    excl <- safelog(sapply(1:ncol(excl), function(x) js.estimate(exp(excl[,x]), wordcounts[x])))
  } 
  freqscore <- apply(logbeta,1,data.table::frank)/ncol(logbeta)
  exclscore <- apply(excl,1,data.table::frank)/ncol(logbeta)
  frex <- 1/(w/freqscore + (1-w)/exclscore)
  apply(frex,2,order,decreasing=TRUE)
}
'''    

def calcfrex(logbeta, w = .5, wordcounts=None):
    excl = (logbeta.T -col_lse(logbeta) ).T
    ncol_excl = len(excl[0])
    ncol= len(logbeta[0])
    if wordcounts is not None:
        #if word counts provided calculate the shrinkage estimator
        
        exclT= excl.T
        for i in range(ncol):
            exclT[i] = safelog(np.apply_along_axis(JSestimate,1,np.exp(exclT[i])), wordcounts[i])
        #wordcounts[x]?
        excl= exclT.T
    freqscore = np.apply_along_axis(ss.rankdata,1,logbeta)/ncol
    exclscore = np.apply_along_axis(ss.rankdata,1,excl)/ncol_excl
    frex = 1.0/(w/freqscore+(1-w)/exclscore)
    np.apply_along_axis(argsort, 0, -1*frex)
    
    

In [12]:
np.argsort(np.array([1,4,3,2]))


array([0, 3, 2, 1])

In [13]:
#corruption dictionary
